# Ejercicio 2: AES

## Apartado a

A continuación se define el cuerpo $R = \mathbb{Z}_2 [x] / (x^8 + x^4 + x^3 + x + 1)$, como indican las especificaciones del algoritmo.

In [1]:
R = PolynomialRing(GF(2), 'x').quotient('x^8 + x^4 + x^3 + x + 1', 'x')
R

Univariate Quotient Polynomial Ring in x over Finite Field of size 2 with modulus x^8 + x^4 + x^3 + x + 1

### Transformaciones entre las distintas formas de representar un byte

In [2]:
# Binario a hexadecimal
def bin_to_hex(binary):
    return hex(int(binary,2))[2:].zfill(2)

# Hexadecimal a binario
def hex_to_bin(hexadecimal):
    return bin(int(hexadecimal, 16))[2:].zfill(8)

# Polinomio correspondiente a un binario
def bin_to_pol(binary):
    binary_list_int = [int(c) for c in binary[::-1]]
    return R(binary_list_int)

# Binario correspondiente a un polinomio
def pol_to_bin(pol):
    return ''.join([str(c) for c in pol.list()[::-1]])

# Hexadecimal correspondiente a un polinomio
def pol_to_hex(pol):
    return bin_to_hex(pol_to_bin(pol))

# Polinomio correspondiente a un hexadecimal
def hex_to_pol(hexadecimal):
    return bin_to_pol(hex_to_bin(hexadecimal))

Definimos la siguiente matriz que se usa en la transformación afín.

In [3]:
M = matrix(Integers(2),8,[
        1,0,0,0,1,1,1,1,
        1,1,0,0,0,1,1,1,
        1,1,1,0,0,0,1,1,
        1,1,1,1,0,0,0,1,
        1,1,1,1,1,0,0,0,
        0,1,1,1,1,1,0,0,
        0,0,1,1,1,1,1,0,
        0,0,0,1,1,1,1,1,
    ])
M

[1 0 0 0 1 1 1 1]
[1 1 0 0 0 1 1 1]
[1 1 1 0 0 0 1 1]
[1 1 1 1 0 0 0 1]
[1 1 1 1 1 0 0 0]
[0 1 1 1 1 1 0 0]
[0 0 1 1 1 1 1 0]
[0 0 0 1 1 1 1 1]

Calculamos la inversa de dicha matriz, que nos será útil para calcular la inversa de la transformación afín.

In [4]:
# Inversa de la matriz M
M_inv = M**(-1)
M_inv

[0 0 1 0 0 1 0 1]
[1 0 0 1 0 0 1 0]
[0 1 0 0 1 0 0 1]
[1 0 1 0 0 1 0 0]
[0 1 0 1 0 0 1 0]
[0 0 1 0 1 0 0 1]
[1 0 0 1 0 1 0 0]
[0 1 0 0 1 0 1 0]

De esta forma, podemos definir siguiente función, `sbox(x,y)`, que dado un byte `xy` sea capaz de aportar la entrada correspondiente a la fila `x` y la columna `y` en la tabla de la Figura 4.9 de los apuntes.

Para ello, se ha calculado la inversa de la transformación afín (4.3) seguida de la toma del inverso multiplicativo en $GF(2^8)$.

In [5]:
def sbox(x,y):
    xy = x + y
    byte_bin = hex_to_bin(xy) # Polinomio correspondiente de xy
    c = vector(Integers(2), [1,1,0,0,0,1,1,0]) # coeficientes del byte {63}
    b = vector(Integers(2), [int(c) for c in byte_bin[::-1]])
    res = M_inv*(b-c) # Inversa de la transformación afín
    
    if (res == 0):
        res = '00'
    else:
        res = bin_to_pol(''.join([str(c) for c in res[::-1]]))
        res = R(res**(-1))
        res = pol_to_hex(res)
    return res

x = '5'
y = '3'
sbox(x,y)

'50'

Luego la posición 43 en la tabla 4.9 viene ocupada por `50`.

## Apartado b

### Matriz de `S-Box inversa`

Obtenemos la matriz que corresponde a S-box inversa, que contiene los valores de sustitución para el byte `xy` (en formato hexadecimal).

In [6]:
hex_digits = [hex(i)[2:] for i in range(16)]
s_box = []

for x in hex_digits:
    row = []
    
    for y in hex_digits:
        row.append(sbox(x,y)) # Calcular la entrada en S-box inversa del byte xy
    
    s_box.append(row)

# Imprimir por pantalla la matriz S-Box inversa    
for row in s_box:
    print(row)

['52', '09', '6a', 'd5', '30', '36', 'a5', '38', 'bf', '40', 'a3', '9e', '81', 'f3', 'd7', 'fb']
['7c', 'e3', '39', '82', '9b', '2f', 'ff', '87', '34', '8e', '43', '44', 'c4', 'de', 'e9', 'cb']
['54', '7b', '94', '32', 'a6', 'c2', '23', '3d', 'ee', '4c', '95', '0b', '42', 'fa', 'c3', '4e']
['08', '2e', 'a1', '66', '28', 'd9', '24', 'b2', '76', '5b', 'a2', '49', '6d', '8b', 'd1', '25']
['72', 'f8', 'f6', '64', '86', '68', '98', '16', 'd4', 'a4', '5c', 'cc', '5d', '65', 'b6', '92']
['6c', '70', '48', '50', 'fd', 'ed', 'b9', 'da', '5e', '15', '46', '57', 'a7', '8d', '9d', '84']
['90', 'd8', 'ab', '00', '8c', 'bc', 'd3', '0a', 'f7', 'e4', '58', '05', 'b8', 'b3', '45', '06']
['d0', '2c', '1e', '8f', 'ca', '3f', '0f', '02', 'c1', 'af', 'bd', '03', '01', '13', '8a', '6b']
['3a', '91', '11', '41', '4f', '67', 'dc', 'ea', '97', 'f2', 'cf', 'ce', 'f0', 'b4', 'e6', '73']
['96', 'ac', '74', '22', 'e7', 'ad', '35', '85', 'e2', 'f9', '37', 'e8', '1c', '75', 'df', '6e']
['47', 'f1', '1a', '71', '1d',